In [3]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import cvxpy as cp
import pandas as pd
from sklearn.model_selection import train_test_split

/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [4]:
TRAIN_DATA = pd.read_csv('studentspen-train.csv')
TEST_DATA=pd.read_csv('studentsdigits-test.csv')

# print(TRAIN_DATA['Digit'].value_counts())

D = 8 # feature dimension

eps = 1e-8 # select support vectors
random.seed(1) # For reproducibility

TRAIN_DATA_NO_LABEL = TRAIN_DATA.drop('Digit', axis=1)

Train_data_normalized=TRAIN_DATA_NO_LABEL.copy()
for column in Train_data_normalized.columns:
    Train_data_normalized[column] = (Train_data_normalized[column] - Train_data_normalized[column].min()) / (Train_data_normalized[column].max() - Train_data_normalized[column].min())  

ONLY_LABELS = TRAIN_DATA[['Digit']]

print(ONLY_LABELS['Digit'].value_counts())
# # ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 0, 'Digit'] = 2

# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 1, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 2, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 3, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 4, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 5, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 6, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 7, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 8, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 9, 'Digit'] = -1
# ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 0, 'Digit'] = 1

# print(ONLY_LABELS['Digit'].value_counts())





# #.........

# TEST_DATA_W_LABELS = TEST_DATA[(TEST_DATA['Digit']==1) | (TEST_DATA['Digit']==7)]

# ONLY_LABELS_TEST = TEST_DATA[['Digit']]

# ONLY_LABELS_TEST.loc[ONLY_LABELS_TEST['Digit'] == 7, 'Digit'] = -1

# TEST_DATA_NO_LABEL = TEST_DATA_W_LABELS.drop('Digit', axis=1)

2    392
4    391
1    390
0    384
5    375
7    374
3    370
8    363
9    357
6    351
Name: Digit, dtype: int64


In [174]:
print(ONLY_LABELS.value_counts())

Digit
-1       3363
 1        384
dtype: int64


In [96]:
# X_train_np=TRAIN_DATA.to_numpy()
# print(X_train_np)
# print(X_train_np.shape)
# y_train_np=ONLY_LABELS.to_numpy()
# print(y_train_np.shape)
# X_test=TEST_DATA_N.to_numpy()
# y_test=ONLY_LABELS_TEST.to_numpy()


In [5]:
X_train, X_validate, y_train, y_validate = train_test_split( 
    Train_data_normalized, ONLY_LABELS, test_size=0.1, shuffle=True,random_state=1)


X_train_np = X_train.values
print(X_train_np)
# X_train_np.shape

y_train_np = y_train.values
# y_train_np.shape

train_data_qty = len(X_train_np)
print(train_data_qty)

[[0.24 0.98 0.78 ... 0.73 0.78 0.43]
 [0.9  0.81 0.68 ... 0.27 0.   0.04]
 [0.8  0.   0.   ... 0.45 0.89 0.78]
 ...
 [0.62 0.69 0.34 ... 0.16 0.12 0.11]
 [0.   0.27 0.42 ... 0.17 1.   0.65]
 [0.83 0.8  0.7  ... 0.24 0.   0.07]]
3372


In [191]:
C = 2 

w = cp.Variable((D, 1))

b = cp.Variable()

epsilon = cp.Variable((train_data_qty, 1))

objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C))

constraints = [cp.multiply(y_train_np, (X_train_np @ w + b)) >= 1 - epsilon,  epsilon >= 0]

prob = cp.Problem(objective, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var w = {}, b = {}".format(w.value, b.value))

status: optimal
optimal value 683.9531276450907
optimal var w = [[-2.04392189e-08]
 [ 2.44858089e-08]
 [ 2.71139297e-08]
 [ 6.82896711e-08]
 [ 1.09146229e-07]
 [ 7.70837428e-08]
 [ 1.07798612e-07]
 [ 2.16200220e-08]], b = 1.0476759056798377


In [6]:
Xt = X_validate.values
# Xt.shape

yt = y_validate.values
# yt.shape



In [193]:
Xt = X_validate.values
# Xt.shape

yt = y_validate.values
# yt.shape

print(yt)
predictions = Xt.dot(w.value) + b.value

print(predictions)

predictions[predictions<0] = -1

predictions[predictions>=0] = 1


count_wrong_predictions = 0
for i in range(len(predictions)):
    if (yt[i] != predictions[i]):
        count_wrong_predictions+=1
        
#count_wrong_predictions

correct_predictions = (len(predictions) - count_wrong_predictions)
#correct_predictions

accuracy = correct_predictions/len(predictions)

print("Accurracy =",accuracy*100,"%")
# Accuracy = 90.19607843137256 %

[[5]
 [6]
 [0]
 [5]
 [0]
 [3]
 [0]
 [4]
 [6]
 [6]
 [5]
 [4]
 [5]
 [8]
 [5]
 [0]
 [3]
 [8]
 [7]
 [1]
 [8]
 [5]
 [5]
 [0]
 [2]
 [6]
 [1]
 [5]
 [2]
 [2]
 [2]
 [5]
 [3]
 [9]
 [4]
 [0]
 [0]
 [7]
 [1]
 [0]
 [8]
 [2]
 [2]
 [1]
 [0]
 [1]
 [3]
 [7]
 [1]
 [7]
 [5]
 [2]
 [2]
 [0]
 [5]
 [1]
 [1]
 [6]
 [1]
 [3]
 [7]
 [4]
 [0]
 [1]
 [4]
 [2]
 [4]
 [3]
 [0]
 [4]
 [2]
 [8]
 [3]
 [5]
 [3]
 [2]
 [7]
 [6]
 [2]
 [7]
 [8]
 [2]
 [6]
 [9]
 [0]
 [9]
 [1]
 [0]
 [8]
 [1]
 [4]
 [2]
 [2]
 [7]
 [0]
 [8]
 [3]
 [1]
 [8]
 [2]
 [4]
 [0]
 [9]
 [4]
 [6]
 [0]
 [3]
 [4]
 [9]
 [5]
 [6]
 [5]
 [6]
 [4]
 [6]
 [8]
 [8]
 [4]
 [9]
 [5]
 [4]
 [6]
 [4]
 [3]
 [7]
 [3]
 [4]
 [1]
 [5]
 [2]
 [4]
 [9]
 [4]
 [3]
 [2]
 [8]
 [1]
 [8]
 [8]
 [0]
 [4]
 [1]
 [2]
 [2]
 [5]
 [4]
 [3]
 [8]
 [6]
 [1]
 [6]
 [9]
 [7]
 [5]
 [1]
 [9]
 [1]
 [7]
 [9]
 [0]
 [0]
 [1]
 [2]
 [0]
 [6]
 [5]
 [1]
 [3]
 [8]
 [2]
 [1]
 [2]
 [9]
 [7]
 [3]
 [5]
 [3]
 [8]
 [5]
 [8]
 [0]
 [6]
 [5]
 [1]
 [5]
 [5]
 [4]
 [9]
 [6]
 [5]
 [5]
 [9]
 [7]
 [6]
 [7]
 [9]
 [2]
 [9]
 [5]
 [8]


In [7]:
w0=np.array([[-1.13659933],
 [-2.50965631],
 [ 0.91862065],
 [-3.22184625],
 [ 1.02545807],
 [ 0.19386996],
 [ 0.25150697],
 [-0.786548  ]])

b0= 0.33543933928742453

w1= np.array([[ 0.11136743],
 [ 1.21448144],
 [ 0.42764079],
 [ 2.29511887],
 [ 0.99617461],
 [-1.05101616],
 [-1.15059   ],
 [ 1.14159889]])

b1= -3.4151083017396027

w2= np.array([[-0.83804506],
 [ 2.61929956],
 [ 2.29061202],
 [-3.15331839],
 [-3.30516777],
 [ 3.98563639],
 [-1.43790671],
 [-3.74383495]])

b2=-0.6081947794777065

w3= np.array([[ 2.36117487],
 [ 1.01028246],
 [ 0.53697785],
 [-3.80501361],
 [ 0.3371793 ],
 [ 4.23653371],
 [ 2.64325668],
 [-4.28420312]])

b3= -4.2447540562542825

w4 = np.array([[-2.40379357e+00],
 [-1.52018826e-01],
 [-5.01215467e-01],
 [ 1.47224059e+00],
 [ 3.66700196e-01],
 [-4.32108217e-01],
 [ 1.46275051e-03],
 [ 1.52510800e+00]])

b4= -1.2605742965221183

w5= np.array([[ 0.26004592],
 [ 2.36151838],
 [-0.73837224],
 [-2.60744333],
 [-0.27603738],
 [ 1.44978346],
 [-0.7101034 ],
 [ 1.24806609]])

b5= -1.4962089198732618

w6= np.array([[ 0.82913454],
 [-2.76922526],
 [-3.40947239],
 [ 1.38989242],
 [ 0.58072053],
 [-3.88156305],
 [ 1.06491686],
 [-4.28774063]])

b6= 1.6284818464197852

w7 = np.array([[ 1.43379355],
 [ 0.35851827],
 [ 0.50105589],
 [ 3.23724083],
 [ 0.75602474],
 [-5.60808442],
 [-0.22440834],
 [ 1.60501467]])

b7 = -3.1665306285027754

w8 = np.array([[-0.07815872],
 [-0.51389246],
 [ 0.42932142],
 [ 0.13262245],
 [-0.63422037],
 [-0.25794879],
 [ 0.1135568 ],
 [-0.04971473]])

b8 = -0.32801820501175705

w9= np.array([[-0.45434529],
 [-0.79645261],
 [-0.84494384],
 [ 1.9420929 ],
 [-0.35394897],
 [ 2.70757961],
 [ 1.57237137],
 [-0.83587514]])

b9= -2.8915283612538984

 

In [114]:
# print(yt)

[[5]
 [6]
 [0]
 [5]
 [0]
 [3]
 [0]
 [4]
 [6]
 [6]
 [5]
 [4]
 [5]
 [8]
 [5]
 [0]
 [3]
 [8]
 [7]
 [1]
 [8]
 [5]
 [5]
 [0]
 [2]
 [6]
 [1]
 [5]
 [2]
 [2]
 [2]
 [5]
 [3]
 [9]
 [4]
 [0]
 [0]
 [7]
 [1]
 [0]
 [8]
 [2]
 [2]
 [1]
 [0]
 [1]
 [3]
 [7]
 [1]
 [7]
 [5]
 [2]
 [2]
 [0]
 [5]
 [1]
 [1]
 [6]
 [1]
 [3]
 [7]
 [4]
 [0]
 [1]
 [4]
 [2]
 [4]
 [3]
 [0]
 [4]
 [2]
 [8]
 [3]
 [5]
 [3]
 [2]
 [7]
 [6]
 [2]
 [7]
 [8]
 [2]
 [6]
 [9]
 [0]
 [9]
 [1]
 [0]
 [8]
 [1]
 [4]
 [2]
 [2]
 [7]
 [0]
 [8]
 [3]
 [1]
 [8]
 [2]
 [4]
 [0]
 [9]
 [4]
 [6]
 [0]
 [3]
 [4]
 [9]
 [5]
 [6]
 [5]
 [6]
 [4]
 [6]
 [8]
 [8]
 [4]
 [9]
 [5]
 [4]
 [6]
 [4]
 [3]
 [7]
 [3]
 [4]
 [1]
 [5]
 [2]
 [4]
 [9]
 [4]
 [3]
 [2]
 [8]
 [1]
 [8]
 [8]
 [0]
 [4]
 [1]
 [2]
 [2]
 [5]
 [4]
 [3]
 [8]
 [6]
 [1]
 [6]
 [9]
 [7]
 [5]
 [1]
 [9]
 [1]
 [7]
 [9]
 [0]
 [0]
 [1]
 [2]
 [0]
 [6]
 [5]
 [1]
 [3]
 [8]
 [2]
 [1]
 [2]
 [9]
 [7]
 [3]
 [5]
 [3]
 [8]
 [5]
 [8]
 [0]
 [6]
 [5]
 [1]
 [5]
 [5]
 [4]
 [9]
 [6]
 [5]
 [5]
 [9]
 [7]
 [6]
 [7]
 [9]
 [2]
 [9]
 [5]
 [8]


In [8]:
Xt = X_validate.values
# Xt.shape

yt = y_validate.values
# yt.shape
meta_pred=[]

wvals=[w0,w1,w2,w3,w4,w5,w6,w7,w8,w9]
bvals=[b0,b1,b2,b3,b4,b5,b6,b7,b8,b9]

for w,b in zip(wvals,bvals):
    predictions = Xt.dot(w) + b
    meta_pred.append(predictions)

In [9]:
meta_pred[9].max()

1.7118905301461012

In [10]:
meta_pred_np=np.array(meta_pred)

In [11]:
meta_pred_np=np.squeeze(meta_pred_np, axis=2)

In [12]:
meta_pred_np.shape

(10, 375)

In [13]:
preds=[]
for j in range(375):
    lol=meta_pred_np[:,j]
    preds.append(np.argmax(lol))
    
        


In [14]:
print(preds)

[5, 6, 0, 5, 8, 3, 0, 4, 6, 2, 9, 4, 5, 8, 8, 0, 3, 8, 7, 2, 9, 9, 5, 0, 2, 6, 1, 5, 1, 7, 2, 5, 3, 9, 4, 0, 0, 7, 3, 0, 7, 2, 2, 1, 6, 1, 3, 1, 1, 1, 5, 2, 2, 0, 9, 1, 2, 6, 9, 3, 7, 4, 0, 9, 6, 2, 4, 1, 0, 4, 2, 1, 3, 5, 3, 2, 1, 6, 2, 7, 7, 2, 6, 9, 0, 9, 9, 0, 0, 1, 4, 2, 2, 1, 0, 0, 3, 7, 7, 2, 4, 0, 9, 4, 6, 0, 3, 4, 9, 5, 6, 5, 6, 4, 6, 2, 1, 4, 9, 5, 5, 6, 4, 3, 7, 3, 4, 1, 5, 2, 4, 9, 4, 3, 2, 7, 9, 8, 7, 0, 4, 1, 2, 2, 5, 9, 3, 3, 6, 1, 6, 9, 3, 5, 4, 9, 1, 7, 3, 0, 0, 7, 1, 0, 6, 7, 1, 3, 0, 2, 1, 2, 9, 7, 3, 7, 3, 1, 5, 1, 6, 6, 9, 1, 9, 9, 0, 9, 6, 8, 5, 9, 7, 6, 7, 9, 2, 9, 5, 5, 1, 6, 1, 9, 0, 6, 9, 7, 0, 4, 1, 1, 9, 7, 9, 5, 5, 5, 7, 2, 3, 0, 1, 6, 1, 2, 2, 0, 2, 6, 4, 2, 6, 0, 5, 7, 6, 3, 3, 7, 5, 2, 6, 2, 5, 5, 2, 6, 1, 7, 6, 2, 5, 9, 7, 7, 9, 4, 7, 4, 5, 0, 5, 4, 0, 5, 9, 5, 2, 7, 1, 2, 9, 5, 1, 1, 1, 6, 3, 1, 3, 3, 5, 7, 4, 2, 3, 6, 7, 0, 9, 3, 4, 0, 4, 4, 4, 6, 3, 7, 4, 0, 3, 9, 3, 0, 4, 0, 3, 3, 4, 6, 1, 0, 6, 4, 2, 6, 0, 9, 2, 5, 2, 4, 6, 4, 9, 2, 2, 5, 0, 7, 2, 

In [15]:
print(lol)

[-0.9638705  -3.14400254 -1.7114376  -2.17239848 -1.75910097 -0.1787848
 -2.99865554 -2.80760931 -0.44491723 -2.35875996]


In [16]:
count_wrong_predictions = 0
for i in range(len(preds)):
    if (yt[i] != preds[i]):
        count_wrong_predictions+=1
        
#count_wrong_predictions

correct_predictions = (len(preds) - count_wrong_predictions)
#correct_predictions

accuracy = correct_predictions/len(preds)

print("Accurracy =",accuracy*100,"%")

Accurracy = 75.46666666666667 %


In [17]:
TEST_DATA = pd.read_csv('studentsdigits-test.csv')

In [18]:
Test_data_normalized=TEST_DATA.copy()

for column in Test_data_normalized.columns:
    Test_data_normalized[column] = (Test_data_normalized[column] - Test_data_normalized[column].min()) / (Test_data_normalized[column].max() - Test_data_normalized[column].min())  

In [21]:
Xt = Test_data_normalized.values
Xt

array([[0.54, 0.68, 0.47, ..., 0.  , 0.13, 0.29],
       [0.68, 0.85, 1.  , ..., 0.8 , 0.9 , 0.53],
       [0.22, 0.03, 0.71, ..., 0.48, 0.82, 1.  ],
       ...,
       [0.24, 0.87, 0.03, ..., 0.51, 0.58, 0.26],
       [0.84, 0.96, 0.72, ..., 0.08, 0.  , 0.  ],
       [0.94, 0.86, 0.7 , ..., 0.11, 0.32, 0.  ]])

In [22]:
meta_pred=[]

wvals=[w0,w1,w2,w3,w4,w5,w6,w7,w8,w9]
bvals=[b0,b1,b2,b3,b4,b5,b6,b7,b8,b9]

for w,b in zip(wvals,bvals):
    predictions = Xt.dot(w) + b
    meta_pred.append(predictions)

In [23]:
meta_pred_np=np.array(meta_pred)

In [24]:
meta_pred_np=np.squeeze(meta_pred_np, axis=2)

In [25]:
meta_pred_np.shape

(10, 3747)

In [28]:
preds=[]
for j in range(3747):
    lol=meta_pred_np[:,j]
    preds.append(np.argmax(lol))

In [29]:
print(preds)

[7, 1, 0, 6, 4, 5, 6, 3, 5, 1, 6, 6, 6, 3, 5, 4, 7, 2, 1, 9, 6, 6, 3, 0, 9, 1, 1, 2, 0, 3, 4, 1, 1, 0, 9, 9, 6, 4, 9, 6, 6, 4, 0, 5, 9, 8, 7, 7, 6, 2, 4, 0, 7, 3, 3, 9, 0, 3, 0, 1, 4, 0, 0, 6, 5, 5, 9, 1, 1, 4, 9, 8, 3, 6, 3, 6, 3, 6, 4, 3, 7, 7, 4, 3, 1, 7, 7, 0, 0, 7, 7, 1, 8, 3, 0, 9, 2, 0, 9, 1, 3, 1, 3, 5, 6, 6, 3, 1, 6, 2, 6, 4, 7, 5, 3, 4, 2, 4, 2, 3, 4, 5, 4, 2, 0, 3, 2, 9, 1, 6, 2, 0, 7, 7, 9, 5, 2, 1, 2, 9, 7, 0, 1, 4, 6, 2, 7, 4, 2, 4, 5, 6, 7, 5, 5, 0, 6, 0, 0, 2, 3, 3, 7, 5, 3, 9, 7, 7, 6, 4, 7, 2, 0, 1, 5, 5, 7, 4, 7, 9, 7, 7, 7, 1, 2, 5, 1, 7, 3, 1, 9, 0, 6, 1, 1, 6, 9, 2, 7, 3, 3, 7, 7, 2, 0, 3, 7, 4, 4, 1, 2, 5, 9, 0, 8, 2, 5, 7, 2, 2, 6, 4, 7, 3, 0, 9, 7, 5, 7, 7, 4, 5, 1, 5, 9, 1, 6, 9, 6, 6, 2, 4, 7, 0, 5, 9, 1, 9, 4, 7, 0, 5, 9, 7, 6, 0, 0, 2, 7, 3, 2, 0, 8, 1, 3, 3, 9, 9, 7, 9, 3, 9, 6, 2, 2, 2, 6, 1, 6, 3, 2, 6, 3, 0, 7, 2, 0, 5, 2, 2, 1, 9, 4, 8, 9, 0, 1, 5, 1, 9, 6, 0, 9, 3, 9, 9, 9, 4, 1, 7, 0, 4, 1, 6, 6, 6, 6, 3, 8, 3, 5, 9, 2, 2, 3, 2, 3, 9, 4, 0, 9, 4, 7, 

In [ ]:
print("\n\n***** Number of Predictions made = ", len(preds))

with open('HADI_digit_prediction_svm.txt', 'w') as file:
    for prediction in preds:
        file.write('%d\n' % prediction)